In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
dev_df=pd.read_csv('./output/dev_statement_section.csv')
dev_df

,statement,section,label
0,there is a 13.2% difference between the result...,"[""Outcome Measurement:"", ""Event-free Survival""...",Contradiction
1,Patients with significantly elevated ejection ...,"[""Inclusion criteria:"", ""Inclusion Criteria:"",...",Contradiction
2,a significant number of the participants in th...,"[""Adverse Events 1:"", ""Total: 20/167 (11.98%)""...",Contradiction
3,the primary trial does not report the PFS or o...,"[""Outcome Measurement:"", ""Local Control Using ...",Entailment
4,Prior treatment with fulvestrant or with a pho...,"[""Inclusion Criteria:"", ""Postmenopausal women ...",Contradiction
...,...,...,...
195,The the primary trial intervention involves on...,"[""INTERVENTION 1:"", ""Letrozole"", ""Participants...",Contradiction
196,the secondary trial reported 1 single case of ...,"[""Adverse Events 1:"", ""Total: 16/48 (33.33%)"",...",Entailment
197,the secondary trial and the primary trial do n...,"[""Outcome Measurement:"", ""Number of Patients W...",Entailment
198,the outcome measurement of the primary trial i...,"[""Outcome Measurement:"", ""Progression-free Sur...",Entailment


In [3]:
hypothesis_lst=dev_df['statement'].values.tolist()
len(hypothesis_lst)

200

In [4]:
evidence_lst=dev_df['section'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

200

In [5]:
label2id={"Contradiction":0,"Entailment":1}
label_lst=dev_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

200

In [6]:
import random
import math
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import T5Tokenizer, T5ForConditionalGeneration

text_tok = T5Tokenizer.from_pretrained("t5-base")
text_clf = T5ForConditionalGeneration.from_pretrained("t5-base")

/home/x20779zz/.local/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [7]:
class InputSequence:
    
    def __init__(self,tok,l_text,l_text2,l_label,batch_size=64,gpu=True,task_prefix = "Detect entailment: "):
        
        self.data_len=len(l_text)
        self.data_idx=[i for i in range(self.data_len)]
        self.texts=tok([task_prefix+' '.join([a,b]) for a,b in zip(l_text,l_text2)],
                       padding="longest",
                       max_length=512,#max_source_length,
                       truncation=True,
                       return_tensors="pt",
                      )
        self.labels=tok(["Entailment" if lab==1 else "Contradiction" for lab in l_label],
                         padding="longest",
                         max_length=128,#,
                         truncation=True,
                         return_tensors="pt",
                        )
        print('tokenize done')
        
        self.batch_size=batch_size
        self.gpu=gpu
        
    def on_epoch_end(self):
        random.shuffle(self.data_idx)
        
    def __getitem__(self,i):
        start=i*self.batch_size
        batch_idx=self.data_idx[start:min(start+self.batch_size,self.data_len)]
        
        return_texts=dict([(k,self.texts[k][batch_idx]) for k in self.texts])
        return_labels=dict([(k,self.labels[k][batch_idx]) for k in self.labels])
        
        if self.gpu:
            return_texts=dict([(k,return_texts[k].cuda()) for k in return_texts])
            return_labels=dict([(k,return_labels[k].cuda()) for k in return_labels])
        
        return return_texts,return_labels
    
    def __len__(self):
        return math.ceil(1.0*self.data_len/self.batch_size)
    

In [8]:
testing_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,batch_size=16,gpu=True)

tokenize done


In [9]:
scores=[]
#T5
model_names=['t5-base']+[
    './output/clf_models/t5-base_epoch_{}.pt'.format(format(epoch,'05d'))
    for epoch in range(10)
]
for model_name in model_names:
    scores.append([])
    clf=T5ForConditionalGeneration.from_pretrained(model_name).cuda()
    with torch.no_grad():
        for batch in range(len(testing_data)):
            batch_texts,batch_labels=testing_data[batch]
            output_texts=text_tok.batch_decode(
                clf.generate(
                    input_ids=batch_texts["input_ids"],
                    attention_mask=batch_texts["attention_mask"],
                    do_sample=False,  # disable sampling to test if batching affects output
                ),
                skip_special_tokens=True
            )
            # print(output_texts)
            scores[-1].append([[0.0,1.0] if t=='Entailment' else [1.0,0.0] for t in output_texts])
            print('model:',model_name,'batch:',batch,end='\r')
    scores[-1]=np.concatenate(scores[-1],axis=0)
    clf.cpu()

/home/x20779zz/.local/lib/python3.9/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
2022-12-09 18:05:35.439208: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


/home/x20779zz/.local/lib/python3.9/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [10]:
from sklearn.metrics import average_precision_score,f1_score,precision_score,recall_score,accuracy_score

y_true=label_lst
results=[]
for epoch in range(len(scores)):
    y_prob=scores[epoch][:,1]
    y_pred=[1 if a>0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch==0 else epoch,
        average_precision_score(y_true,y_prob),
        f1_score(y_true,y_pred),
        precision_score(y_true,y_pred),
        recall_score(y_true,y_pred),
        accuracy_score(y_true,y_pred)
    ])

import pandas as pd

pd.DataFrame(results,columns=['epoch','AVG_PREC','F1','PREC','REC','ACC'])

/home/x20779zz/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,epoch,AVG_PREC,F1,PREC,REC,ACC
0,pretrained,0.500000,0.000000,0.000000,0.00,0.500
1,1,0.497518,0.580913,0.496454,0.70,0.495
2,2,0.486154,0.404494,0.461538,0.36,0.470
3,3,0.500000,0.621212,0.500000,0.82,0.500
4,4,0.500000,0.603175,0.500000,0.76,0.500
5,5,0.492881,0.352201,0.474576,0.28,0.485
6,6,0.508000,0.331034,0.533333,0.24,0.515
7,7,0.495119,0.445652,0.488095,0.41,0.490
8,8,0.502536,0.414201,0.507246,0.35,0.505
9,9,0.518737,0.532663,0.535354,0.53,0.535


from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

task_prefix = "translate English to German: "
# use different length sentences to test batching
sentences = ["The house is wonderful.", "I like to work in NYC."]

inputs = tokenizer([task_prefix + sentence for sentence in sentences], return_tensors="pt", padding=True)

output_sequences = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    do_sample=False,  # disable sampling to test if batching affects output
)

print(tokenizer.batch_decode(output_sequences, skip_special_tokens=True))